### conda uninstall tensorflow
### conda uninstall h5py

### pip install tensorflow
### pip install keras
### pip install h5py
### pip install --upgrade scikit-learn
### pip install --upgrade pandas


In [80]:
import os
import cv2
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from tqdm.notebook import tqdm
from sklearn.model_selection import GroupKFold, train_test_split

In [81]:
%cd C:/Users/4Kims/0_python/x_ray

C:\Users\4Kims\0_python\x_ray


In [82]:
df = pd.read_csv('C:/Users/4Kims/0_python/x_ray/train.csv')

In [83]:
df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN


In [84]:
class_names = df.sort_values(by=['class_id']).class_name.unique()
class_dict = pd.Series(class_names).to_dict()
class_dict

{0: 'Aortic enlargement',
 1: 'Atelectasis',
 2: 'Calcification',
 3: 'Cardiomegaly',
 4: 'Consolidation',
 5: 'ILD',
 6: 'Infiltration',
 7: 'Lung Opacity',
 8: 'Nodule/Mass',
 9: 'Other lesion',
 10: 'Pleural effusion',
 11: 'Pleural thickening',
 12: 'Pneumothorax',
 13: 'Pulmonary fibrosis',
 14: 'No finding'}

In [85]:
df_Abn = df

In [86]:
df_Abn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67914 entries, 0 to 67913
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   image_id    67914 non-null  object 
 1   class_name  67914 non-null  object 
 2   class_id    67914 non-null  int64  
 3   rad_id      67914 non-null  object 
 4   x_min       36096 non-null  float64
 5   y_min       36096 non-null  float64
 6   x_max       36096 non-null  float64
 7   y_max       36096 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 4.1+ MB


In [87]:
df_Abn.isnull().sum()

image_id          0
class_name        0
class_id          0
rad_id            0
x_min         31818
y_min         31818
x_max         31818
y_max         31818
dtype: int64

In [88]:
# df_Abn.dropna(subset=['x_min'], inplace=True)
df_Abn.fillna(0,inplace=True)

In [89]:
# df_Abn.drop_duplicates(subset=['image_id','class_id'], keep='first',inplace=True)
# df_Abn.drop_duplicates(['image_id', 'class_id'], keep='first',inplace=True)

In [90]:
df_Abn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67914 entries, 0 to 67913
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   image_id    67914 non-null  object 
 1   class_name  67914 non-null  object 
 2   class_id    67914 non-null  int64  
 3   rad_id      67914 non-null  object 
 4   x_min       67914 non-null  float64
 5   y_min       67914 non-null  float64
 6   x_max       67914 non-null  float64
 7   y_max       67914 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 4.1+ MB


In [91]:
# df_Abn.drop(['index'], axis = 1)

In [92]:
# df_Abn.reset_index(level=None, drop=False,inplace=True,col_level=0,col_fill='')
# df_Abn.reset_index(drop=True,inplace=True)


In [93]:
df_bb_cnt = df_Abn['x_min'].count()
df_bb_cnt

67914

In [94]:
df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,0.0,0.0,0.0,0.0
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,0.0,0.0,0.0,0.0
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,0.0,0.0,0.0,0.0


In [95]:
df_Abn.loc[0:0,'image_id']

0    50a418190bc3fb1ef1633bf9678929b3
Name: image_id, dtype: object

In [96]:
csv_img_id = df_Abn.loc[0:0,'image_id'].to_string()
print(csv_img_id.split('    ')[1])

50a418190bc3fb1ef1633bf9678929b3


In [97]:
print(df_Abn.loc[:,'image_id'][0])

50a418190bc3fb1ef1633bf9678929b3


In [98]:
class_id = df_Abn.loc[0:0,'class_id'].to_string().split('    ')[1]
print(class_id)

14


In [99]:
%cd C:/Users/4Kims/0_python/x_ray/data/images/

C:\Users\4Kims\0_python\x_ray\data\images


In [100]:
# curr_dir = os.getcwd()
# files = os.listdir(curr_dir)

# for file in files:
#     image_name = file.split(".")[0]
#     print(image_name)

In [101]:
def bounting_box_cal(x1,y1,x2,y2,img_w,img_h):
    
    xmin = x1
    ymin = y1
    xmax = x2
    ymax = y2
    
    print(f' xmin: {xmin} /  ymin: {ymin} / xmax: {xmax} / ymax: {ymax}')

    # Transform the bbox co-ordinates as per the format required by YOLO v5
    b_center_x = (xmin + xmax)/2 
    b_center_y = (ymin + ymax)/2
    b_width    =  xmax - xmin
    b_height   =  ymax - ymin
    
    print(f' b_center_x: {b_center_x} /  b_center_y: {b_center_y} / b_width: {b_width} / b_height: {b_height}')

    image_w = img_w
    image_h = img_h     

    # Normalise the co-ordinates by the dimensions of the image
    b_center_x /= image_w 
    b_center_y /= image_h 
    b_width    /= image_w 
    b_height   /= image_h 

    return b_center_x, b_center_y, b_width, b_height

In [102]:
from PIL import Image

curr_dir = os.getcwd()
files = os.listdir(curr_dir)
txt_path = 'C:/Users/4Kims/0_python/x_ray/data/labels/'
iter_num = 0
for file in files: 
    print_buffer = []
    idx = 0
    image = Image.open(file)
#     image.show()
    image_name = file.split(".")[0]
    img_w, img_h = image.size 
    csv_img_id = df_Abn.loc[:,'image_id']
    class_id = df_Abn.loc[:,'class_id']
    x1 = df_Abn.loc[:,'x_min']
    y1 = df_Abn.loc[:,'y_min']
    x2 = df_Abn.loc[:,'x_max']
    y2 = df_Abn.loc[:,'y_max']

#     print(image_name[0]," png 파일 사이즈는 : " ,imag1_size)
    for idx in range(df_bb_cnt):
        # csv_img_id = df_Abn.loc[idx:idx,'image_id'].to_string().split('    ')[1]
        if csv_img_id[idx] == image_name:
            print(f'인덱스 {idx} 이미지 id {csv_img_id[idx]} 이미지이름 {image_name}')
            # df_Abn['img_yes'] = 1
            if class_id[idx] == 14:
                # x_ctr_norm = 0
                # y_ctr_norm = 0
                # width = 1/img_w
                # height = 1/img_h
                # print_buffer.append("{} {:.5f} {:.5f} {:.5f} {:.5f}".format(class_id[idx], x_ctr_norm, y_ctr_norm, width, height))
                break
            else:
                x_ctr_norm, y_ctr_norm, width, height = bounting_box_cal(float(x1[idx]),float(y1[idx]),float(x2[idx]),float(y2[idx]),img_w,img_h)
                print_buffer.append("{} {:.5f} {:.5f} {:.5f} {:.5f}".format(class_id[idx], x_ctr_norm, y_ctr_norm, width, height))
    image.close()
    print(iter_num)
    print("\n".join(print_buffer), file= open(txt_path+image_name + ".txt", "w"))

    iter_num += 1



인덱스 11114 이미지 id 00053190460d56c53cc3e57321387478 이미지이름 00053190460d56c53cc3e57321387478
0
인덱스 664 이미지 id 0005e8e3701dfb1dd93d53e2ff537b6e 이미지이름 0005e8e3701dfb1dd93d53e2ff537b6e
 xmin: 900.0 /  ymin: 587.0 / xmax: 1205.0 / ymax: 888.0
 b_center_x: 1052.5 /  b_center_y: 737.5 / b_width: 305.0 / b_height: 301.0
인덱스 13648 이미지 id 0005e8e3701dfb1dd93d53e2ff537b6e 이미지이름 0005e8e3701dfb1dd93d53e2ff537b6e
 xmin: 932.0 /  ymin: 567.0 / xmax: 1197.0 / ymax: 896.0
 b_center_x: 1064.5 /  b_center_y: 731.5 / b_width: 265.0 / b_height: 329.0
인덱스 15180 이미지 id 0005e8e3701dfb1dd93d53e2ff537b6e 이미지이름 0005e8e3701dfb1dd93d53e2ff537b6e
 xmin: 900.0 /  ymin: 587.0 / xmax: 1205.0 / ymax: 888.0
 b_center_x: 1052.5 /  b_center_y: 737.5 / b_width: 305.0 / b_height: 301.0
인덱스 16938 이미지 id 0005e8e3701dfb1dd93d53e2ff537b6e 이미지이름 0005e8e3701dfb1dd93d53e2ff537b6e
 xmin: 905.0 /  ymin: 583.0 / xmax: 1203.0 / ymax: 890.0
 b_center_x: 1054.0 /  b_center_y: 736.5 / b_width: 298.0 / b_height: 307.0
인덱스 48751 이미지 id 0005e8

In [103]:
# for idx in range(10):
#     print(idx)

In [104]:
# from PIL import Image

# curr_dir = os.getcwd()
# files = os.listdir(curr_dir)
# val_txt_path = '/home/class3/x_ray/data/labels/val/'
# for file in files:
#     print_buffer = []
#     idx = 0
#     class_id = 14
#     x_ctr_norm = 0
#     y_ctr_norm = 0 
#     width = 0
#     height = 0
#     image = Image.open(file)
# #     image.show()
#     image_name = file.split(".")[0]
#     imag1_size = image.size

# #     print(image_name[0]," png 파일 사이즈는 : " ,imag1_size)
#     while idx < df_bb_cnt:
#         csv_img_id = df_Abn.loc[idx:idx,'image_id'].to_string().split('    ')[1]
#         if csv_img_id == image_name:
#             print(f'인덱스 {idx} 이미지 id {csv_img_id} 이미지이름 {image_name}')
#             df_Abn['img_yes'] = 1
#             x1 = df_Abn.loc[idx:idx,'x_min'].to_string().split('    ')[1]
#             y1 = df_Abn.loc[idx:idx,'y_min'].to_string().split('    ')[1]
#             x2 = df_Abn.loc[idx:idx,'x_max'].to_string().split('    ')[1]
#             y2 = df_Abn.loc[idx:idx,'y_max'].to_string().split('    ')[1]
#             img_w = imag1_size[0]
#             img_h = imag1_size[1]
#             x_ctr_norm, y_ctr_norm, width, height = bounting_box_cal(float(x1),float(y1),float(x2),float(y2),img_w,img_h)
# #             df_Abn['x_ctr_norm'] = x_ctr_norm
# #             df_Abn['y_ctr_norm'] = y_ctr_norm
# #             df_Abn['width_norm'] = width
# #             df_Abn['height_norm'] = height
#             class_id = df_Abn.loc[idx:idx,'class_id'].to_string().split('    ')[1]
            
#             break
#         idx += 1
#     print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, x_ctr_norm, y_ctr_norm, width, height))
#     image.close
#     print("\n".join(print_buffer), file= open(val_txt_path+image_name + ".txt", "w"))